Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = ""
COLLABORATORS = ""

---

<!--NOTEBOOK_HEADER-->
*This notebook contains material from [PyRosetta](https://RosettaCommons.github.io/PyRosetta.notebooks);
content is available [on Github](https://github.com/RosettaCommons/PyRosetta.notebooks.git).*

<!--NAVIGATION-->
< [Protein Design 2](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/06.04-Protein-Design-2.ipynb) | [Contents](toc.ipynb) | [Index](index.ipynb) | [*De Novo* Parametric Backbone Design](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/06.06-Introduction-to-Parametric-backbone-design.ipynb) ><p><a href="https://colab.research.google.com/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/06.05-HBNet-Before-Design.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a>

# HBNet Before Design

Keywords: HBNet, OperateOnResidueSubset, getPoseExtraScore, InterGroupInterfaceByVectorSelector, ChainSelector, PreventRepackingRLT, RestrictToRepackingRLT, OperateOnResidueSubset, ResiduePDBInfoHasLabelSelector, PackRotamersMover

Sometimes in Rosetta we want to run implicit multistage design. That is, we want to optimize one conformation while implicitly modeling another (either negatively or positively). There are many ways to accomplish this depending on your interests. In this section we will look at HBNet, a tool for explicitly designing hydrogen bond networks.

One negative-design approach is to implicitly model binding specificity. Designing a complicated network of hydrogen bonds at one interface will implicitly destabilize other interfaces. Hydrogen bonds are so sensitive to geometry that competing interfaces are unlikely to be able to "satisfy" the network well enough to remain competetive.

The previous example can also be viewed through the implicit positive-design lens as well. We often find that Rosetta designs very hydrophobic interfaces (especially with newer score functions). Running HBNet before the traditional design protocols can boost the polar residue concentration of your interface in exchange for a small cost packing quality. In other words, we can implicitly stabilize the unbound state by running HBNet, but it might mildly destabilize the bound state.

Our experience shows that it is useful to run both with and without HBNet, depending on your design case. It is possible that the default design protocol handles your implicits states well enough. When that fails, though, there is not much to do to fix it other than to run pre-design protocols like HBNet. An added benefit of HBNet is that it can provide "seeds" for packing, which can influence design diversity if nothing else.

In [ ]:
# Notebook setup
import sys
if 'google.colab' in sys.modules:
    !pip install pyrosettacolabsetup
    import pyrosettacolabsetup
    pyrosettacolabsetup.setup()
    print ("Notebook is set for PyRosetta use in Colab.  Have fun!")

**Make sure you are in the directory with the pdb files:**

`cd google_drive/My\ Drive/student-notebooks/`

In [ ]:
# From previous section:
from pyrosetta import *
from pyrosetta.teaching import *
pyrosetta.init("-mute core -mute basic")
print( "init complete" )

We prepare for HBNet the same way that we prepare for packing. We setup the pose and score function as before...

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

Just like before, you can edit the resfile to your own personal specifications. Alternatively, you can use task operations to automate the process. Let's use task operations to fix all residues not at the interface.

## Setting Designable Residues:

Create a new task for design


In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

## Running HBNet

This is an interface case so we will use HBNetStapleInterface.  We will use both the code-level interface, and the XML interface as an introduction to this functionality.  The XML interface to PyRosetta will be covered more in later workshops.  

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

Wait, my score is terrible.

__Question:__ Why?

## Finishing Design:

Well of course the score is terrible, the pose is dense with clashes. We had 116 packable residues and only assigned states to 5 of them. The other 111 residues are still in their input conformations and likely clash with the 5 we just assigned.

We need to run the packer (either using PackRotamersMover or FastDesign) but we don't want to overwrite the residues we just assigned with HBNet. The trick here is to select the residues with "HBNet" labels and fix them.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

## We made it...?

The change in score is a better, but still positive. One great thing about HBNet is that it can return multiple poses. Each one is slightly worse than the previous by HBNet's standards but might design into something better. Let's try a few to see if they help.

In [ ]:
#there were 10 (or so) networks total, but let's just try the next 5
#this might take a few minutes...
if not os.getenv('DEBUG'): #Adding this line to decrease runtime on the testing server
    for x in range(0,5):
        extra_pose = hbnet.get_additional_output()
        if extra_pose is None:
            break
        task_design3 = task_factory.create_task_and_apply_taskoperations( extra_pose )
        task_design3.or_linmem_ig( True )
        pack_mover = PackRotamersMover( scorefxn, task_design3 )
        pack_mover.apply( extra_pose )
        print( "Change in score", scorefxn(extra_pose) - scorefxn(start_pose) )

## But wait, there's more??

Great! We found a few results that designed to me more stable than the input pose (-60, -45, and -31 REU)!

The main score function is not the only way to evaluate these networks. HBNet also adds its own score terms. These are useful for sorting/filtering decoys before running expensive packing calculations.

In [ ]:
from pyrosetta.rosetta.core.pose import hasPoseExtraScore, getPoseExtraScore

if hasPoseExtraScore( pose, "HBNet_NumUnsatHpol" ):
    #All 3 of these metrics are explained in more detail in Maguire, Boyken, et al. (see second reference below)

    #NumUnsatHpol is HBNet's primary sorting metric, it counts the number of polar hydrogen atoms that are unsatisfied (buried and not forming hbonds). We know that there are no heavy (non-hydrogen) unsatisfied atoms because HBNet filters those out by default. Lower is better
    print( "HBNet_NumUnsatHpol", getPoseExtraScore( pose, "HBNet_NumUnsatHpol" ) )

    #HBNet's secondary sorting metric. 1.0 if every polar atom in the network is forming the maximum number of hbonds. Higher is better
    print( "HBNet_Saturation", getPoseExtraScore( pose, "HBNet_Saturation" ) )

    #HBNet's tertiary sorting metric. A little complicated but lower is better.
    print( "HBNet_Score", getPoseExtraScore( pose, "HBNet_Score" ) )
else:
    print( "Somebody go bug a developer to enable this feature for PyRosetta" )

## Advice for using this in the wild

#### ex1 ex2

HBonds are very sensitive to sidechain sampling resolution. I highly recommend using -ex1 and -ex2. You can do this by adding:
```py
ex1ex2 = ExtraRotamersGeneric()
ex1ex2.ex1( True )
ex1ex2.ex2( True )
task_factory.push_back( ex1ex2 )
```

#### get_additional_output

As we saw in the exercise, the first result out of HBNet does not always wind up being the best. Try designing with a few results from `hbnet.get_additional_output()` to get more coverage of the design space. For the commandline users reading this, this functionality can also be accessed via `multistage_rosetta_scripts` or the `MultiplePoseMover` in `rosetta_scripts`. See the `rosetta_scripts_scripts` repository for examples.

#### set_monte_carlo_seed_must_be_buried

I highly recommend playing with the `set_monte_carlo_seed_must_be_buried` mentioned above. Without it, HBNet tends to just design many surface networks that nobody really cares about.

## Thought Question

The energy of HBNet+Design is often less favorable that the energy after an equivalent design run without HBNet. Why do people still use HBNet?


## References


- Boyken SE, Chen Z, Groves B, et al. De novo design of protein homo-oligomers with modular hydrogen-bond network-mediated specificity. Science. 2016;352(6286):680–687. doi:10.1126/science.aad8865


- Maguire JB, Boyken SE, Baker D, Kuhlman B. Rapid Sampling of Hydrogen Bond Networks for Computational Protein Design. J Chem Theory Comput. 2018;14(5):2751–2760. doi:10.1021/acs.jctc.8b00033

<!--NAVIGATION-->
< [Protein Design 2](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/06.04-Protein-Design-2.ipynb) | [Contents](toc.ipynb) | [Index](index.ipynb) | [*De Novo* Parametric Backbone Design](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/06.06-Introduction-to-Parametric-backbone-design.ipynb) ><p><a href="https://colab.research.google.com/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/06.05-HBNet-Before-Design.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a>